# Facility Location Problem
In this session, we will solve the facility location problem. The purpose of this problem is to minimize installation costs and demand allocation from distribution centers to customers (stores). The data used comes from the Optimization Mathematics course that I took when I was in college with Mr. Setyo Tri Windras Mara. I will complete this model using the help of the PuLP library.

**Problem:**

A retail company plans to penetrate the market to Kulonprogo Regency because of the potential for NYIA Airport. The company intends to open ten stores in the Kulonprogo area and they are currently evaluating three potential locations to build a distribution center.

### 1. Importing Library

In [1]:
import pandas as pd
import numpy as np
from pulp import *

### 2.Importing Data
The data used has an excel file type with 3 sheets. Therefore we use pd.read_excel to import the data. I added some parameters such as "headers" and "usecols" as well as transpose (.T) on the *Cost* sheet for easy data access.

In [2]:
dist_center = pd.read_excel("Facility Location Problem.xlsx", sheet_name="Distribution centers").set_index("Locations")
customers = pd.read_excel("Facility Location Problem.xlsx", sheet_name="Customers").set_index("Customers")
costs = pd.read_excel("Facility Location Problem.xlsx", sheet_name="Costs", header=1, usecols=range(1,12)).rename(columns={"Unnamed: 1":"Sites"}).set_index("Sites").T

#### a. Distribution Center
This table lists the costs required to construct the distribution center as well as the storage capacity of the facility.

In [3]:
dist_center

,Cost,Capacity
Locations,,
1,300,250
2,200,150
3,500,300


#### b. Customers
This table contains ten stores that will be opened in Kulonprogo Regency and their demand per period.

In [4]:
customers

,Demand per Periode
Customers,
1,30
2,20
3,40
4,50
5,40
6,60
7,30
8,50
9,40


#### c. Costs
This table contains the costs required to distribute goods from the distribution center to the stores.

In [5]:
costs

Sites,1,2,3
1,96,76,58
2,97,66,68
3,100,83,100
4,81,52,52
5,89,54,83
6,57,63,74
7,71,96,66
8,96,99,64
9,55,94,63
10,84,86,83


### 3. Add Variables
In addition to adding variables, I also made several parameters used, namely:
- list_customer: store collection
- list_sites: a collection of distribution centers
- keys: store and distribution center combination for variable creation purposes

The variables used in this problem are:
- yij: demand allocation from the store i which is fulfilled by distribution center j. This variable is an integer greater than 0.
- xij: binary type variable that states the condition of the distribution center will be built or not

<img src="https://raw.githubusercontent.com/rianromad/Image-Storage/main/flp%20var.PNG?token=AOWKXL6HY4EP5HD63CJVEYTBBKI6G" />

*Source: Sarker and Newton - Optimization Modelling, A Practical Approach*

In [6]:
#create parameter
list_customers = customers.index
list_sites = dist_center.index
keys = [(i,j) for i in list_customers for j in list_sites]

#create variable
y = LpVariable.dicts("y", keys, 0, None, "Integer")
x = LpVariable.dicts("x", list_sites, 0, None, "Binary")

### 4. Initiate the Model
Since the goal of this problem is to minimize costs, use **LpMinimize**

In [7]:
model = LpProblem("Facility_Location_Problem", LpMinimize)

### 5. Add Objective Function and Constraints

**Objective Function:**

The purpose of this problem is to minimize the costs as well as the costs required to distribute goods from the distribution center to the store.

**Constraints:**
 
 - Customer/store demand (Di) must be met
 - Store i cannot be served by distribution center j unless the facility is built and the number of goods distributed from distribution center j to all stores must be less than equal to its capacity (Uj).
 
<img src="https://raw.githubusercontent.com/rianromad/Image-Storage/main/flp.PNG?token=AOWKXL5AYCONTRB5GHHZHRTBBKHX4" />

*Source: Sarker and Newton - Optimization Modelling, A Practical Approach*

In [8]:
#fungsi tujuan
model += lpSum([dist_center.loc[j,"Cost"]*x[j] for j in list_sites]) +  lpSum([costs.loc[i,j]*y[i,j] for (i,j) in keys])

In [9]:
#constraint
for i in list_customers:
    model += lpSum([y[i,j] for j in list_sites]) == customers.loc[i,"Demand per Periode"]
    
for j in list_sites:
    model += lpSum([y[i,j] - dist_center.loc[j,"Capacity"]*x[j] for i in list_customers]) <=0

### 6. Solve the Model and Check the Result

In [10]:
#Check model status (1 = optimal)
model.solve()

1

In [11]:
#Check how many cost which spent
model.objective.value()

26260

I also created a table containing the solutions to the problems above to make them easier to reach.

In [12]:
solutions = []
for j in list_sites:
    for i in list_customers:
        solutions.append(y[i,j].varValue)

solutions_grouped = [solutions[i:i+10] for i in range(0,len(solutions),10)]
df_sol = pd.DataFrame(solutions_grouped, columns=list_customers, index=list_sites)
df_sol

Customers,1,2,3,4,5,6,7,8,9,10
Locations,,,,,,,,,,
1,0,0,0,0,0,60,0,0,40,0
2,0,20,40,50,40,0,0,0,0,0
3,30,0,0,0,0,0,30,50,0,40


In [13]:
#export tabel
df_sol.to_excel("FLP_Solution.xlsx")

# Conclusion

From the table above, the ten stores get supplies from three distribution centers. No shop doesn't get a share. The second facility has a lower price with a capacity of 150 so that it is fully utilized without waste. Unlike the first and third facilities, which still have remaining capacity. This is a short tutorial to solve the facility location problem using PuLP, sorry if there are mistakes or shortcomings. Good luck :)